In [5]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import os
from supervised.automl import AutoML

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

## Firstly, let's load in our data and convert to CSV

I try automl with numbered features to predict strains now. I also use imputed target variables now instead of our targets with NaN present.

In [6]:
data = pd.read_excel('data/FrontiersDS.xlsx', sheet_name='Wide')

In [12]:
imputed = pd.read_csv('data/imputed_wide.csv', index_col='Unnamed: 0')

In [13]:
features = pd.read_csv('data/features_wide.csv', index_col='Unnamed: 0')

In [29]:
x = features['Strain']
exit_status_map = {'SA_A': 0, 'SA_B': 1 ,'EC_A': 2, 'PA_A': 3, 'EC_B': 4, 'PA_B': 5}
xyz = x.map(exit_status_map)

In [30]:
xyz

3     0
4     0
5     0
6     0
7     0
     ..
82    4
83    4
84    4
85    4
86    4
Name: Strain, Length: 84, dtype: int64

In [33]:
X_train, X_test, y_train, y_test = train_test_split(imputed, xyz, test_size=0.15)

In [34]:
automl = AutoML(algorithms=["Decision Tree", "Linear", "Random Forest", "Nearest Neighbors", "Xgboost"], total_time_limit=5*60)
automl.fit(X_train, y_train)

AutoML directory: AutoML_11
The task is multiclass_classification with evaluation metric logloss
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Random Forest', 'Nearest Neighbors', 'Xgboost']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 2 models
1_DecisionTree logloss 0.599139 trained in 30.84 seconds
2_Linear logloss 0.34405 trained in 20.31 seconds
* Step default_algorithms will try to check up to 3 models
3_Default_Xgboost logloss 0.116492 trained in 34.56 seconds
4_Default_RandomForest logloss 0.67374 trained in 23.93 seconds


X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
X does not have valid feature names, but KNeighborsClassifier was fitted with feature names


Exception while producing SHAP explanations. 'NoneType' object has no attribute 'shap_values'
Continuing ...
5_Default_NearestNeighbors logloss 1.252451 trained in 4.52 seconds
* Step ensemble will try to check up to 1 model
Ensemble logloss 0.116492 trained in 0.4 seconds
AutoML fit time: 149.7 seconds
AutoML best model: 3_Default_Xgboost


AutoML(algorithms=['Decision Tree', 'Linear', 'Random Forest',
                   'Nearest Neighbors', 'Xgboost'],
       total_time_limit=300)

In [35]:
y_predicted = automl.predict(X_test)
y_predicted = pd.DataFrame(y_predicted, columns={'Strain'})
print(pd.DataFrame({"Predicted": y_predicted["Strain"], "Target": np.array(y_test)}))

    Predicted  Target
0           2       2
1           4       4
2           0       1
3           4       4
4           3       3
5           2       2
6           4       4
7           2       2
8           2       2
9           1       1
10          5       5
11          1       1
12          3       3


In [41]:
def accuracy_auto(array_pred, array_target):
    i = 0
    count = 0
    zx = np.array(array_target)
    while i < len(array_pred):
        if array_pred['Strain'][i] == zx[i]:
            count += 1
            i += 1
        else:
            i += 1

    return(count / len(array_pred))

In [42]:
accuracy_auto(y_predicted, y_test)

0.9230769230769231

In [50]:
# Cross-Validation for 15:85 split
from sklearn import svm
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
clf.score(X_test, y_test)

0.46153846153846156

In [43]:
X_train30, X_test30, y_train30, y_test30 = train_test_split(imputed, xyz, test_size=0.30)

In [44]:
automl = AutoML(algorithms=["Decision Tree", "Linear", "Random Forest", "Nearest Neighbors", "Xgboost"], total_time_limit=5*60)
automl.fit(X_train30, y_train30)

AutoML directory: AutoML_12
The task is multiclass_classification with evaluation metric logloss
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Random Forest', 'Nearest Neighbors', 'Xgboost']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 2 models
1_DecisionTree logloss 0.386876 trained in 34.19 seconds
2_Linear logloss 0.094014 trained in 20.64 seconds
* Step default_algorithms will try to check up to 3 models
3_Default_Xgboost logloss 0.049986 trained in 27.98 seconds
4_Default_RandomForest logloss 0.777886 trained in 31.48 seconds


X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
X does not have valid feature names, but KNeighborsClassifier was fitted with feature names


Exception while producing SHAP explanations. 'NoneType' object has no attribute 'shap_values'
Continuing ...
5_Default_NearestNeighbors logloss 0.550311 trained in 4.41 seconds
* Step ensemble will try to check up to 1 model
Ensemble logloss 0.049986 trained in 0.37 seconds
AutoML fit time: 157.78 seconds
AutoML best model: 3_Default_Xgboost


AutoML(algorithms=['Decision Tree', 'Linear', 'Random Forest',
                   'Nearest Neighbors', 'Xgboost'],
       total_time_limit=300)

In [45]:
y_predicted30 = automl.predict(X_test30)
y_predicted30 = pd.DataFrame(y_predicted30, columns={'Strain'})
print(pd.DataFrame({"Predicted": y_predicted30["Strain"], "Target": np.array(y_test30)}))

    Predicted  Target
0           4       4
1           0       0
2           1       1
3           4       4
4           0       0
5           3       5
6           0       0
7           0       0
8           3       3
9           5       5
10          3       3
11          0       0
12          0       1
13          1       0
14          2       2
15          4       4
16          5       5
17          3       3
18          0       0
19          5       5
20          2       2
21          1       1
22          2       2
23          0       1
24          2       2
25          3       3


In [46]:
accuracy_auto(y_predicted30, y_test30)

0.8461538461538461

In [49]:
# Cross-Validation for 30:70 split
from sklearn import svm
clf30 = svm.SVC(kernel='linear', C=1).fit(X_train30, y_train30)
clf30.score(X_test30, y_test30)

0.7692307692307693

In [51]:
X_train40, X_test40, y_train40, y_test40 = train_test_split(imputed, xyz, test_size=0.40)

In [52]:
automl = AutoML(algorithms=["Decision Tree", "Linear", "Random Forest", "Nearest Neighbors", "Xgboost"], total_time_limit=5*60)
automl.fit(X_train40, y_train40)

AutoML directory: AutoML_13
The task is multiclass_classification with evaluation metric logloss
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Random Forest', 'Nearest Neighbors', 'Xgboost']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 2 models
1_DecisionTree logloss 0.565435 trained in 33.87 seconds
2_Linear logloss 0.061669 trained in 20.66 seconds
* Step default_algorithms will try to check up to 3 models
3_Default_Xgboost logloss 0.086053 trained in 29.81 seconds
4_Default_RandomForest logloss 0.650587 trained in 23.67 seconds


X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
X does not have valid feature names, but KNeighborsClassifier was fitted with feature names


Exception while producing SHAP explanations. 'NoneType' object has no attribute 'shap_values'
Continuing ...
5_Default_NearestNeighbors logloss 0.387216 trained in 4.51 seconds
* Step ensemble will try to check up to 1 model
Ensemble logloss 0.061669 trained in 0.35 seconds
AutoML fit time: 151.07 seconds
AutoML best model: 2_Linear


AutoML(algorithms=['Decision Tree', 'Linear', 'Random Forest',
                   'Nearest Neighbors', 'Xgboost'],
       total_time_limit=300)

In [53]:
y_predicted40 = automl.predict(X_test40)
y_predicted40 = pd.DataFrame(y_predicted40, columns={'Strain'})
print(pd.DataFrame({"Predicted": y_predicted40["Strain"], "Target": np.array(y_test40)}))

    Predicted  Target
0           4       4
1           0       0
2           1       1
3           4       4
4           0       0
5           3       5
6           3       0
7           3       0
8           3       3
9           5       5
10          3       3
11          2       0
12          0       1
13          1       0
14          2       2
15          4       4
16          5       5
17          5       3
18          0       0
19          5       5
20          0       2
21          1       1
22          2       2
23          1       1
24          2       2
25          3       3
26          5       5
27          5       5
28          1       0
29          1       1
30          5       3
31          0       1
32          3       3
33          0       2


In [54]:
accuracy_auto(y_predicted40, y_test40)

0.6470588235294118

In [56]:
# Cross-Validation for 40:60 split
from sklearn import svm
clf40 = svm.SVC(kernel='linear', C=1).fit(X_train40, y_train40)
clf40.score(X_test40, y_test40)

0.6470588235294118

In [57]:
X_train50, X_test50, y_train50, y_test50 = train_test_split(imputed, xyz, test_size=0.50)

In [58]:
automl = AutoML(algorithms=["Decision Tree", "Linear", "Random Forest", "Nearest Neighbors", "Xgboost"], total_time_limit=5*60)
automl.fit(X_train50, y_train50)

AutoML directory: AutoML_14
The task is multiclass_classification with evaluation metric logloss
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Random Forest', 'Nearest Neighbors', 'Xgboost']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 2 models
1_DecisionTree logloss 0.020865 trained in 23.47 seconds
2_Linear logloss 0.061263 trained in 19.54 seconds
* Step default_algorithms will try to check up to 3 models
3_Default_Xgboost logloss 0.051239 trained in 23.53 seconds
4_Default_RandomForest logloss 0.627161 trained in 24.0 seconds


X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
X does not have valid feature names, but KNeighborsClassifier was fitted with feature names


Exception while producing SHAP explanations. 'NoneType' object has no attribute 'shap_values'
Continuing ...
5_Default_NearestNeighbors logloss 0.446942 trained in 4.81 seconds
* Step ensemble will try to check up to 1 model
Ensemble logloss 0.020865 trained in 0.41 seconds
AutoML fit time: 133.95 seconds
AutoML best model: 1_DecisionTree


AutoML(algorithms=['Decision Tree', 'Linear', 'Random Forest',
                   'Nearest Neighbors', 'Xgboost'],
       total_time_limit=300)

In [59]:
y_predicted50 = automl.predict(X_test50)
y_predicted50 = pd.DataFrame(y_predicted50, columns={'Strain'})
print(pd.DataFrame({"Predicted": y_predicted50["Strain"], "Target": np.array(y_test50)}))

    Predicted  Target
0           4       4
1           1       0
2           1       1
3           4       4
4           0       0
5           3       5
6           0       0
7           0       0
8           3       3
9           5       5
10          3       3
11          0       0
12          0       1
13          1       0
14          2       2
15          4       4
16          5       5
17          5       3
18          0       0
19          5       5
20          1       2
21          1       1
22          4       2
23          0       1
24          2       2
25          3       3
26          5       5
27          5       5
28          0       0
29          1       1
30          5       3
31          1       1
32          3       3
33          1       2
34          3       3
35          3       5
36          2       2
37          1       0
38          0       0
39          4       4
40          4       2
41          2       2


In [60]:
accuracy_auto(y_predicted50, y_test50)

0.6904761904761905

In [61]:
# Cross-Validation for 50:50 split
from sklearn import svm
clf50 = svm.SVC(kernel='linear', C=1).fit(X_train50, y_train50)
clf50.score(X_test50, y_test50)

0.5952380952380952

In [62]:
X_train25, X_test25, y_train25, y_test25 = train_test_split(imputed, xyz, test_size=0.25)

In [63]:
automl = AutoML(algorithms=["Decision Tree", "Linear", "Random Forest", "Nearest Neighbors", "Xgboost"], total_time_limit=5*60)
automl.fit(X_train25, y_train25)

AutoML directory: AutoML_15
The task is multiclass_classification with evaluation metric logloss
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Random Forest', 'Nearest Neighbors', 'Xgboost']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 2 models
1_DecisionTree logloss 0.840242 trained in 24.98 seconds
2_Linear logloss 0.330437 trained in 20.24 seconds
* Step default_algorithms will try to check up to 3 models
3_Default_Xgboost logloss 0.089317 trained in 25.17 seconds
4_Default_RandomForest logloss 0.722411 trained in 23.44 seconds


X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
X does not have valid feature names, but KNeighborsClassifier was fitted with feature names


Exception while producing SHAP explanations. 'NoneType' object has no attribute 'shap_values'
Continuing ...
5_Default_NearestNeighbors logloss 1.335658 trained in 4.52 seconds
* Step ensemble will try to check up to 1 model
Ensemble logloss 0.089317 trained in 0.34 seconds
AutoML fit time: 138.93 seconds
AutoML best model: 3_Default_Xgboost


AutoML(algorithms=['Decision Tree', 'Linear', 'Random Forest',
                   'Nearest Neighbors', 'Xgboost'],
       total_time_limit=300)

In [64]:
y_predicted25 = automl.predict(X_test25)
y_predicted25 = pd.DataFrame(y_predicted25, columns={'Strain'})
print(pd.DataFrame({"Predicted": y_predicted25["Strain"], "Target": np.array(y_test25)}))

    Predicted  Target
0           2       4
1           0       0
2           1       1
3           4       4
4           0       0
5           3       5
6           0       0
7           0       0
8           3       3
9           5       5
10          3       3
11          0       0
12          0       1
13          1       0
14          2       2
15          4       4
16          5       5
17          3       3
18          0       0
19          5       5
20          2       2


In [65]:
accuracy_auto(y_predicted25, y_test25)

0.8095238095238095

In [67]:
# Cross-Validation for 25:75 split
from sklearn import svm
clf25 = svm.SVC(kernel='linear', C=1).fit(X_train25, y_train25)
clf25.score(X_test25, y_test25)

0.7142857142857143